In [3]:
import os
%cd ..
os.getcwd()

/Users/macbook/Documents/Code/game


'/Users/macbook/Documents/Code/game'

In [45]:
import pygame
import importlib

import src.animation
import src.character
import src.player
import src.ai_character

importlib.reload(src.animation)
importlib.reload(src.character)
importlib.reload(src.player)
importlib.reload(src.ai_character)

from src.animation import Animation
from src.character import Character
from src.player import PlayerCharacter
from src.ai_character import AICharacter

from src.utils import load_dict_from_folder

In [46]:
pygame.init()
WIDTH, HEIGHT = 1280, 720
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()
running = True
player_x, player_y = WIDTH // 2, HEIGHT // 2
player_size = 50 # 50 x 50
image_path = os.path.join("images", "backgrounds", "volcano.png")

background = pygame.image.load(image_path)
background = pygame.transform.scale(background, (WIDTH, HEIGHT))


In [ ]:
player_dict = load_dict_from_folder("images/characters/mark")
player_dict

In [49]:
player = PlayerCharacter(100, 100, speed=2, sprite_dict=player_dict)
enemy1 = AICharacter(600, 400, speed=2, sprite_dict=player_dict)
enemy2 = AICharacter(500, 400, speed=2, sprite_dict=player_dict)

In [50]:
all_agents = pygame.sprite.Group()
enemy_group = pygame.sprite.Group()
all_agents.add(player, enemy1, enemy2)
enemy_group.add(enemy1, enemy2)

In [ ]:

background = pygame.image.load("images/backgrounds/good_back.jpg")
background = pygame.transform.scale(background, screen.get_size())

running = True
while running:
    screen.blit(background, (0, 0))  
    keys = pygame.key.get_pressed()
    events = pygame.event.get()
    
    for agent in all_agents:
        collisions = pygame.sprite.spritecollide(agent, all_agents, False)
        collisions = [coll for coll in collisions if coll != agent]
    # collided_enemies = pygame.sprite.spritecollide(player, enemy_group, False)
        if collisions:
            # Collision detected
            print("Collision detected!")
            # For example, if the player is attacking, damage the enemy.
            if agent.is_attacking:
                for col in collisions:
                    col.take_damage(10)
    
    for event in events:
        if event.type == pygame.QUIT:
            running = False
    enemy_group.update()
    player.update(keys, events, enemy_group)
    # print(f"Position: {player.x, player.y}")
    for agent in all_agents:
        agent.draw(screen)
        if agent.is_dead:
            agent.kill()
  
    pygame.display.flip()
    clock.tick(60) 
